<a href="https://colab.research.google.com/github/sqhang/ClarifAI/blob/main/Evaluation_scripts_for_all_jsons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First you must upload all json files to this runtime from the following google drive folder: [click here](https://drive.google.com/drive/folders/13y3SDOsFPXfsIR3KN61LvUC_rUM7r7vA?usp=sharing) 

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.1 MB/s eta 0:0

In [ ]:
# Load a validation dataset with reference answers.
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json

--2023-04-27 22:43:56--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.111.153, 185.199.109.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4854279 (4.6M) [application/json]
Saving to: ‘dev-v1.1.json.1’

dev-v1.1.json.1     100%[===================>]   4.63M  --.-KB/s    in 0.02s   

2023-04-27 22:43:56 (187 MB/s) - ‘dev-v1.1.json.1’ saved [4854279/4854279]



In [ ]:
import json

with open("dev-v1.1.json", "r") as file:
    dev_json = json.load(file)

In [ ]:
dev_json["data"][1]["title"]

'Warsaw'

In [ ]:
with open("ref_answers.json", "r") as file:
    ref_answers = json.load(file)

In [ ]:
ref_answers[5][1]

[{'answer_start': 32, 'text': 'cultures'},
 {'answer_start': 91, 'text': 'numeracy'},
 {'answer_start': 101, 'text': 'craftsmanship'},
 {'answer_start': 187, 'text': 'life skills'}]

In [ ]:
with open("gpt4_replies_with_4khz_context.json", "r") as file:
    data = json.load(file)

In [ ]:
data[4][1]

['Culture', 'Numeracy', 'Craftsmanship', 'Life skills']

In [ ]:
from evaluate import load

# This is the evaluation script foar all datasets.
def compute_scores(json_name, dev_json):
  with open(json_name, "r") as file:
    context_4khz_answers = json.load(file)

  squad_metric = load("squad")
  article_stats = []
  for art_idx in range(len(context_4khz_answers)):
    par_stats = []
    for par_idx in range(len(context_4khz_answers[art_idx])):
      idx = f"{art_idx}_{par_idx}"
      predictions = []
      references = []
      for ans_idx in range(len(context_4khz_answers[art_idx][par_idx])):
        ref_qas = dev_json["data"][art_idx + 1]["paragraphs"][par_idx]["qas"][ans_idx]
        id = ref_qas["id"]
        references.append({'answers': ref_qas["answers"], 'id': id})
        predictions.append({'prediction_text': context_4khz_answers[art_idx][par_idx][ans_idx], 'id': id})
      results = squad_metric.compute(predictions=predictions, references=references)
      par_stats.append(results)
    print(f"Article {art_idx + 1} par stats: {par_stats}")
    article_em = sum(elem["exact_match"] for elem in par_stats)/len(par_stats)
    article_f1 = sum(elem["f1"] for elem in par_stats)/len(par_stats)
    article_stats.append({"exact_match": article_em, "f1": article_f1})
  total_em = sum(elem["exact_match"] for elem in article_stats)/len(article_stats)
  total_f1 = sum(elem["f1"] for elem in article_stats)/len(article_stats)
  print(f"Total EM = {total_em}; Total F1 = {total_f1}")
  print("Article Stats:")
  for art_idx in range(len(article_stats)):
    em = article_stats[art_idx]["exact_match"]
    f1 = article_stats[art_idx]["f1"]
    print(f"article #{art_idx + 1}: EM = {em}; F1 = {f1}")

In [ ]:
# Compute scores for gpt4 for (1-3) and gpt3.5 for (3-5) with reference context.
compute_scores("gpt4_replies_with_ref_context.json", dev_json)

Article 1 par stats: [{'exact_match': 100.0, 'f1': 100.0}, {'exact_match': 80.0, 'f1': 93.33333333333333}, {'exact_match': 80.0, 'f1': 93.33333333333333}, {'exact_match': 100.0, 'f1': 100.0}, {'exact_match': 100.0, 'f1': 100.0}, {'exact_match': 80.0, 'f1': 80.0}, {'exact_match': 50.0, 'f1': 78.82352941176471}, {'exact_match': 60.0, 'f1': 91.11111111111111}, {'exact_match': 60.0, 'f1': 70.0}, {'exact_match': 60.0, 'f1': 77.14285714285714}, {'exact_match': 60.0, 'f1': 83.33333333333334}, {'exact_match': 80.0, 'f1': 93.33333333333333}, {'exact_match': 60.0, 'f1': 89.33333333333334}, {'exact_match': 100.0, 'f1': 100.0}, {'exact_match': 100.0, 'f1': 100.0}, {'exact_match': 80.0, 'f1': 93.33333333333333}, {'exact_match': 100.0, 'f1': 100.0}, {'exact_match': 80.0, 'f1': 93.33333333333333}, {'exact_match': 100.0, 'f1': 100.0}, {'exact_match': 60.0, 'f1': 92.53333333333333}, {'exact_match': 60.0, 'f1': 89.33333333333334}, {'exact_match': 80.0, 'f1': 84.44444444444444}, {'exact_match': 80.0, 'f1

In [ ]:
# Compute scores for gpt3.5 for (1-5) with reference context.
compute_scores("gpt3.5_replies_with_ref_context.json", dev_json)

Total EM = 94.0; Total F1 = 98.16666666666666
Article Stats:
article #1: EM = 90.0; F1 = 97.5
article #2: EM = 100.0; F1 = 100.0
article #3: EM = 80.0; F1 = 93.33333333333333
article #4: EM = 100.0; F1 = 100.0
article #5: EM = 100.0; F1 = 100.0


In [ ]:
# Compute scores for gpt3.5 for high-quality audios (44.1khz).
compute_scores("gpt4_replies_with_44.1khz_context.json", dev_json)

starting article: 1
starting article: 2
starting article: 3
starting article: 4
starting article: 5
Total EM = 63.0; Total F1 = 72.80555555555556
Article Stats:
article #1: EM = 70.0; F1 = 74.44444444444444
article #2: EM = 50.0; F1 = 62.5
article #3: EM = 80.0; F1 = 93.33333333333333
article #4: EM = 75.0; F1 = 93.75
article #5: EM = 40.0; F1 = 40.0


In [ ]:
# Compute scores for gpt3.5 for low-quality audios (4khz).
compute_scores("gpt4_replies_with_4khz_context.json", dev_json)

starting article: 1
starting article: 2
starting article: 3
starting article: 4
starting article: 5
Total EM = 43.0; Total F1 = 59.33333333333333
Article Stats:
article #1: EM = 60.0; F1 = 68.0
article #2: EM = 25.0; F1 = 37.5
article #3: EM = 60.0; F1 = 91.99999999999999
article #4: EM = 50.0; F1 = 79.16666666666666
article #5: EM = 20.0; F1 = 20.0
